#tHIS IS FOR THE MANUALL PROCESS FOR CREATING THE STREAM

# creating the folders to upload the files using the Notebook commands

In [0]:
dbutils.fs.mkdirs('/Volumes/workspace/raw/rawvolume/rawdata')

In [0]:
dbutils.fs.mkdirs('/Volumes/workspace/raw/rawvolume/rawdata/bookings')
dbutils.fs.mkdirs('/Volumes/workspace/raw/rawvolume/rawdata/flights')
dbutils.fs.mkdirs('/Volumes/workspace/raw/rawvolume/rawdata/customers')
dbutils.fs.mkdirs('/Volumes/workspace/raw/rawvolume/rawdata/airports')

##Create Volumes via commands

In [0]:
%sql
create volume workspace.raw.bronze;
create volume workspace.raw.silver;
create volume workspace.raw.gold;

## Creating the dedicated Scehmas for the relevent layers


In [0]:
%sql
create schema workspace.bronze;
create schema workspace.silver;
create schema workspace.gold;

In [0]:
dbutils.fs.rm('/Volumes/workspace/raw/bronze')
dbutils.fs.rm('/Volumes/workspace/raw/silver')
dbutils.fs.rm('/Volumes/workspace/raw/gold')

## Creating the volumes in the different schemas

In [0]:
%sql
create volume workspace.bronze.bronzevolume;
create volume workspace.silver.silvervolume;
create volume workspace.gold.goldvolume;

#Incremental Data Ingestion

In [0]:
df = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "csv")\
    .option("cloudFiles.schemaLocation", "/Volumes/workspace/bronze/bronzevolume/bookings/checkpoint")\
    .option("cloudFiles.schemaEvolutionMode", "rescue")\
    .load("/Volumes/workspace/raw/rawvolume/rawdata/bookings")

# this is used to write or save the read data from the raw to the bronze layer

In [0]:
df.writeStream.format("delta")\
    .outputMode("append")\
    .trigger(once=True)\
    .option("checkpointLocation", "/Volumes/workspace/bronze/bronzevolume/bookings/checkpoint")\
    .option("path", "/Volumes/workspace/bronze/bronzevolume/bookings/data")\
    .start()

In [0]:
%sql
SELECT * FROM DELTA.`/Volumes/workspace/bronze/bronzevolume/bookings/data/`

In [0]:
%sql
SELECT * FROM DELTA.`/Volumes/workspace/bronze/bronzevolume/airports/data/`

In [0]:
%sql
SELECT * FROM DELTA.`/Volumes/workspace/bronze/bronzevolume/flights/data/`

In [0]:
%sql
SELECT * FROM DELTA.`/Volumes/workspace/bronze/bronzevolume/customers/data/`